In [1]:
pip install pycryptodome

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from os import urandom

## A nice property satisfied by DES

We will check, with a concrete example, that: 

<strong>$$ \mathbf{{\rm NOT(DES(m, k)) = DES(NOT(m), NOT(k)) }} $$</strong>






This identity:

* Only applies to the DES cipher
* Only holds for ECB mode
* Does not extend to 3DES
* Requires a block-aligned, unpadded message

Here’s a test script that:

* Generates a random 8-byte key and 8-byte plaintext.

* Computes DES(m, k)

* Computes NOT(DES(m, k))

* Compares it to DES(NOT(m), NOT(k))

In [3]:
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes

def bytes_to_bitstring(data: bytes) -> str:
    return ''.join(f'{b:08b}' for b in data)

def bitstring_to_bytes(bitstr: str) -> bytes:
    return int(bitstr, 2).to_bytes(len(bitstr) // 8, byteorder='big')

def flip_bits(data: bytes) -> bytes:
    return bytes(~b & 0xFF for b in data)

def des_encrypt_ecb(key: bytes, plaintext: bytes) -> bytes:
    cipher = DES.new(key, DES.MODE_ECB)
    return cipher.encrypt(plaintext)

# === Main check ===
if __name__ == "__main__":
    # Use 8-byte aligned data (no padding)
    m = get_random_bytes(8)
    k = get_random_bytes(8)

    print("Original message     :", bytes_to_bitstring(m))
    print("Original key         :", bytes_to_bitstring(k))

    # DES(m, k)
    c = des_encrypt_ecb(k, m)
    not_c = flip_bits(c)

    # DES(NOT(m), NOT(k))
    not_m = flip_bits(m)
    not_k = flip_bits(k)
    c2 = des_encrypt_ecb(not_k, not_m)

    print("\nDES(m, k)            :", bytes_to_bitstring(c))
    print("NOT(DES(m, k))       :", bytes_to_bitstring(not_c))
    print("DES(NOT(m), NOT(k))  :", bytes_to_bitstring(c2))

    print("\nTest passed?         :", not_c == c2)


Original message     : 0111001100001001010000110100101011110100111100000011101111011011
Original key         : 1110000110000001100011111101110011011110010010100000000010010001

DES(m, k)            : 1000010010011010111011010111100010001111010110101110010101101011
NOT(DES(m, k))       : 0111101101100101000100101000011101110000101001010001101010010100
DES(NOT(m), NOT(k))  : 0111101101100101000100101000011101110000101001010001101010010100

Test passed?         : True


In [4]:
BLOCK_SIZE_BITS = 64  # TripleDES uses 64-bit blocks

def bytes_to_bitstring(data: bytes) -> str:
    return ''.join(f'{b:08b}' for b in data)

def pad_data(data: bytes) -> bytes:
    padder = padding.PKCS7(BLOCK_SIZE_BITS).padder()
    return padder.update(data) + padder.finalize()

def unpad_data(padded_data: bytes) -> bytes:
    unpadder = padding.PKCS7(BLOCK_SIZE_BITS).unpadder()
    return unpadder.update(padded_data) + unpadder.finalize()

def des3_encrypt(key: bytes, iv: bytes, plaintext: bytes) -> bytes:
    cipher = Cipher(algorithms.TripleDES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padded = pad_data(plaintext)
    return encryptor.update(padded) + encryptor.finalize()

def des3_decrypt(key: bytes, iv: bytes, ciphertext: bytes) -> bytes:
    cipher = Cipher(algorithms.TripleDES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()
    return unpad_data(decrypted_padded)

def generate_3des_key() -> bytes:
    return urandom(24)  # 24 bytes = 192-bit key (3-key 3DES)

def generate_iv() -> bytes:
    return urandom(8)  # Block size = 8 bytes for DES/3DES

# === MAIN TEST ===
if __name__ == "__main__":
    message = b"A un amigo perdido"

    print("Original plaintext (bytes):", message)
    print("Original plaintext (bits): ", bytes_to_bitstring(message))

    key = generate_3des_key()
    iv = generate_iv()

    ciphertext = des3_encrypt(key, iv, message)
    decrypted = des3_decrypt(key, iv, ciphertext)

    print("\n--- 3DES with cryptography ---")
    print("Key (bits):         ", bytes_to_bitstring(key))
    print("IV (bits):          ", bytes_to_bitstring(iv))
    print("Ciphertext (bits):  ", bytes_to_bitstring(ciphertext))
    print("Decrypted (bits):   ", bytes_to_bitstring(decrypted))
    print("Decrypted (text):   ", decrypted)


Original plaintext (bytes): b'A un amigo perdido'
Original plaintext (bits):  010000010010000001110101011011100010000001100001011011010110100101100111011011110010000001110000011001010111001001100100011010010110010001101111

--- 3DES with cryptography ---
Key (bits):          110111110011010111010111111010101101100100101111000110010010001001111110111101011111010101101001100110001010000100111010000100111100110100011101100110010110001100001010110010001101001110011011
IV (bits):           0111000000001111000101000101010110011100001001011011110001001001
Ciphertext (bits):   111101111111111001000011000110100011111111001110111001001101101101111000010011101101110110000110010010000011111000110000000000010010101101001010000000000001111110011110100101101010000011110111
Decrypted (bits):    010000010010000001110101011011100010000001100001011011010110100101100111011011110010000001110000011001010111001001100100011010010110010001101111
Decrypted (text):    b'A un amigo perdido'
